In [1]:
!pip install -U kaleido

import gradio as gr
import plotly.graph_objects as go
import pandas as pd

# Load the CSV files
hazard_0_path = r'Hazard_0.csv'
survivor_0_path = r'Survivor_0.csv'
hazard_1_path = r'Hazard_1.csv'
survivor_1_path = r'Survivor_1.csv'
hazard_ratio_path = r'HazardRatio.csv'

try:
    hazard_0_data = pd.read_csv(hazard_0_path)
    survivor_0_data = pd.read_csv(survivor_0_path)
    hazard_1_data = pd.read_csv(hazard_1_path)
    survivor_1_data = pd.read_csv(survivor_1_path)
    hazard_ratio_data = pd.read_csv(hazard_ratio_path)
    print("Data loaded successfully")
except FileNotFoundError as e:
    print(f"File not found: {e}")
    hazard_0_data = survivor_0_data = hazard_1_data = survivor_1_data = hazard_ratio_data = None

# Function to plot hazard rates with HDI bounds
def plot_hazard_data(hazard_data, title):
    if hazard_data is None:
        raise ValueError("Hazard data not loaded")

    required_columns = {'time', 'mean', 'hdi_5.5%', 'hdi_94.5%'}
    if not required_columns.issubset(hazard_data.columns):
        raise ValueError("Required columns not found in hazard data")

    time = hazard_data['time']
    hazard_mean = hazard_data['mean']
    hdi_5_5 = hazard_data['hdi_5.5%']
    hdi_94_5 = hazard_data['hdi_94.5%']

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=time, y=hazard_mean, mode='lines', name='Best Guess',
        line=dict(color='black'),
        hovertemplate="Time: %{x}<br>Best Guess: %{y}<extra></extra>"
    ))
    fig.add_trace(go.Scatter(
        x=time, y=hdi_94_5, mode='lines', name='HDI 94.5%',
        line=dict(color='magenta', dash='dash'),
        hovertemplate="Time: %{x}<br>HDI 94.5%: %{y}<extra></extra>"
    ))
    fig.add_trace(go.Scatter(
        x=time, y=hdi_5_5, mode='lines', name='HDI 5.5%',
        line=dict(color='magenta', dash='dash'),
        hovertemplate="Time: %{x}<br>HDI 5.5%: %{y}<extra></extra>"
    ))

    fig.update_layout(
        title=title,
        xaxis_title='Time',
        yaxis_title='Hazard Rate',
        hovermode='x'
    )

    return fig

# Function to plot survivor rates with HDI bounds
def plot_survivor_data(survivor_data, title):
    if survivor_data is None:
        raise ValueError("Survivor data not loaded")

    required_columns = {'time', 'mean', 'hdi_5.5%', 'hdi_94.5%'}
    if not required_columns.issubset(survivor_data.columns):
        raise ValueError("Required columns not found in survivor data")

    time = survivor_data['time']
    survivor_mean = survivor_data['mean']
    hdi_5_5 = survivor_data['hdi_5.5%']
    hdi_94_5 = survivor_data['hdi_94.5%']

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=time, y=survivor_mean, mode='lines', name='Best Guess',
        line=dict(color='black'),
        hovertemplate="Time: %{x}<br>Best Guess: %{y}<extra></extra>"
    ))
    fig.add_trace(go.Scatter(
        x=time, y=hdi_94_5, mode='lines', name='HDI 94.5%',
        line=dict(color='magenta', dash='dash'),
        hovertemplate="Time: %{x}<br>HDI 94.5%: %{y}<extra></extra>"
    ))
    fig.add_trace(go.Scatter(
        x=time, y=hdi_5_5, mode='lines', name='HDI 5.5%',
        line=dict(color='magenta', dash='dash'),
        hovertemplate="Time: %{x}<br>HDI 5.5%: %{y}<extra></extra>"
    ))

    fig.update_layout(
        title=title,
        xaxis_title='Time',
        yaxis_title='Survivor Rate',
        hovermode='x'
    )

    return fig

# Function to update plot based on selected dataset
def update_plot(choice):
    try:
        if choice == "Data0":
            hazard_plot = plot_hazard_data(hazard_0_data, "Hazard_0 Data")
            survivor_plot = plot_survivor_data(survivor_0_data, "Survivor_0 Data")
            return hazard_plot, survivor_plot
        elif choice == "Data1":
            hazard_plot = plot_hazard_data(hazard_1_data, "Hazard_1 Data")
            survivor_plot = plot_survivor_data(survivor_1_data, "Survivor_1 Data")
            return hazard_plot, survivor_plot
        elif choice == "DataRatio":
            hazard_plot = plot_hazard_data(hazard_ratio_data, "HazardRatio Data")
            return hazard_plot, None
        else:
            return None, None
    except Exception as e:
        return f"Error: {e}", None

# Function to save the current plot as an image in JPG format
def save_plot(choice):
    try:
        if choice == "Data0":
            hazard_plot = plot_hazard_data(hazard_0_data, "Hazard_0 Data")
            hazard_plot.write_image("Hazard_0 Data.jpg", format="jpg")
            survivor_plot = plot_survivor_data(survivor_0_data, "Survivor_0 Data")
            survivor_plot.write_image("Survivor_0 Data.jpg", format="jpg")
            return "Hazard_0 Data saved as Hazard_0 Data.jpg", "Survivor_0 Data saved as Survivor_0 Data.jpg"
        elif choice == "Data1":
            hazard_plot = plot_hazard_data(hazard_1_data, "Hazard_1 Data")
            hazard_plot.write_image("Hazard_1 Data.jpg", format="jpg")
            survivor_plot = plot_survivor_data(survivor_1_data, "Survivor_1 Data")
            survivor_plot.write_image("Survivor_1 Data.jpg", format="jpg")
            return "Hazard_1 Data saved as Hazard_1 Data.jpg", "Survivor_1 Data saved as Survivor_1 Data.jpg"
        elif choice == "DataRatio":
            hazard_plot = plot_hazard_data(hazard_ratio_data, "HazardRatio Data")
            hazard_plot.write_image("HazardRatio Data.jpg", format="jpg")
            return "HazardRatio Data saved as HazardRatio Data.jpg"
        else:
            return "No plot to save."
    except Exception as e:
        return f"Error saving plot: {e}"

# Function to clear the plot area
def clean():
    return None, None, None  # Returns blank outputs to clear both plots

# Define Gradio interface with buttons for each function
with gr.Blocks() as demo:
    gr.Markdown("# Hazard/Survivor Plot Selector")

    with gr.Row():
        select_dataset = gr.Radio(["Data0", "Data1", "DataRatio"], label="Select Dataset")
        clean_button = gr.Button("Clear")
        plot_button = gr.Button("Generate")
        save_button = gr.Button("Save")
    
    with gr.Row():
        hazard_plot_area = gr.Plot(label="Hazard Plot")
        survivor_plot_area = gr.Plot(label="Survivor Plot")
        
    save_status = gr.Textbox(label="Saved Status", interactive=True)

    # Define interactions
    plot_button.click(
        fn=update_plot, 
        inputs=select_dataset, 
        outputs=[hazard_plot_area, survivor_plot_area]
    )
    clean_button.click(
        fn=clean, 
        outputs=[hazard_plot_area, survivor_plot_area,save_status]
    )
    save_button.click(
        fn=save_plot, 
        inputs=select_dataset, 
        outputs=save_status
    )

print("Launching Gradio interface...")
demo.launch(share=False)
print("Gradio interface launched.")

/opt/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Data loaded successfully
Launching Gradio interface...
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Gradio interface launched.
